# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [7]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [13]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [14]:
# confirm that our new database was created
# 1.enter MongoDB console: use uk_food
# 2.enter MongoDB console : createCollection("establishments")
# 3.import json to this database by enter this command in terminal:
# mongoimport --type json -d uk_food -c establishments --jsonArray Resources/establishments.json
pprint(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [15]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [16]:
# review the collections in our new database
pprint(db.list_collection_names())

['establishments']


In [17]:
# review the collections in our new database
collections = db.list_collection_names()

# Print the collections
pprint("Collections in 'uk_food' database:")
for collection in collections:
    pprint(collection)

"Collections in 'uk_food' database:"
'establishments'


In [18]:
# review a document in the establishments collection
establishments_collection = db['establishments']
document = establishments_collection.find_one()

# Pretty print the document
print("Document in the 'establishments' collection:")
pprint(document)

Document in the 'establishments' collection:
{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66516625a5bca31b3ca53cf8'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self'}],
 'm

In [21]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [22]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [23]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant_data)


In [25]:
# Check that the new restaurant was inserted
pprint(result)

InsertOneResult(ObjectId('66516af722ddb521c0ecfe6c'), acknowledged=True)


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [33]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_result = establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessType": 1,"BusinessTypeID": 1})

# Print the results
pprint(business_type_result)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('66516625a5bca31b3ca53cf9')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [34]:
# Update the new restaurant with the correct BusinessTypeID
# Extract the BusinessTypeID from the result
business_type_id = business_type_result["BusinessTypeID"]

# Define the dictionary with the update
update_data = {
    "$set": {
        "BusinessTypeID": business_type_id
    }
}

update_result = establishments.update_one({"BusinessName": "Penang Flavours"}, update_data)

In [35]:
# Confirm that the new restaurant was updated
result = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66516af722ddb521c0ecfe6c'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [38]:
# Find how many documents have LocalAuthorityName as "Dover"
count = establishments.count_documents({"LocalAuthorityName": "Dover"})
pprint("Number of documents with LocalAuthorityName as 'Dover':")
pprint(count)

"Number of documents with LocalAuthorityName as 'Dover':"
994


In [39]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

# Print the number of deleted documents
print("Number of documents deleted:", delete_result.deleted_count)

Number of documents deleted: 994


In [40]:
# Check if any remaining documents include Dover
count = establishments.count_documents({"LocalAuthorityName": "Dover"})
pprint("Number of documents remain with LocalAuthorityName as 'Dover':")
pprint(count)

"Number of documents with LocalAuthorityName as 'Dover':"
0


In [41]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

# Print the remaining document
print("One remaining document after deletion:")
pprint(remaining_document)

One remaining document after deletion:
{'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'ChangesByServerID': 0,
 'Distance': 4591.871474174837,
 'FHRSID': 289353,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6QH',
 'RatingDate': '2016-09-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66516625a5bca31b3ca53fda'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'},
 'links': [{'href': 'https://api.ratings.food

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [42]:
# Change the data type from String to Decimal for longitude and latitude
result = establishments.update_many({}, [
    {"$set": {
        "geocode.latitude": {"$toDouble": "$geocode.latitude"}, 
        "geocode.longitude": {"$toDouble": "$geocode.longitude"}
    }}
])

print("Number of documents updated:", result.modified_count)

Number of documents updated: 38786


Use `update_many` to convert `RatingValue` to integer numbers.

In [43]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [45]:
# Change the data type from String to Integer for RatingValue
result = establishments.update_many({}, [
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
])
print("Number of documents updated:", result.modified_count)


Number of documents updated: 0
Data type of RatingValue field: <class 'int'>


In [48]:
# Check that the coordinates and rating value are now numbers

document = establishments.find_one()
# Print the data type of RatingValue in the found document
print("Data type of RatingValue field:", type(document['RatingValue']))
print("Data type of geocode[latitude] field:", type(document['geocode']['latitude']))
print("Data type of geocode[longitude] field:", type(document['geocode']['longitude']))

Data type of RatingValue field: <class 'int'>
Data type of geocode[latitude] field: <class 'float'>
Data type of geocode[longitude] field: <class 'float'>
